In [14]:
import networkx as nx

In [41]:
def shortest_path_lengths(graph):
    '''Returns a dictionary of dictionaries. dict[i][j] contains the distance
    between vertices i and j.'''
    dist_gen = nx.shortest_path_length(graph)

    node_distances = {}
    for node, distance_dict in dist_gen:
        node_distances[node] = distance_dict
        
    return node_distances

def find_farthest(node_distances, marked):
    '''Find the unmarked vertex farthest from the root (assumes root is  vertex 0)'''
    farthest = None
    for node, dist in node_distances[0].items():
        if node not in marked and (farthest is None or dist > node_distances[0][farthest]):
            farthest = node
    return farthest
        

def burn_tree(tree):
    '''Implementation for tree burning algorithm
    Input:  a tree to burn
    Output: a burning sequence for the tree
    '''
    centers = []
    marked = Set()
    
    # Calculate distance between all pairs of nodes
    node_distances = shortest_path_length(tree)
    
    i = 0
    while True:
        # Find the unmarked vertex farthest from the root (assumes root is  vertex 0)
        farthest = find_farthest(node_distances, marked)
        
        

In [42]:
balanced_tree = nx.balanced_tree(r=2, h=5)
print(balanced_tree.nodes)
print(balanced_tree.degree(0))
print(balanced_tree.degree(62))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
2
1


In [43]:
marked = set()

dist_gen = nx.shortest_path_length(balanced_tree)

node_distances = {}
for node, distance_dict in dist_gen:
    node_distances[node] = distance_dict

print(find_farthest(node_distances, marked))
print(balanced_tree.degree(31))

# After removing vertex 31, it should find vertex 32
balanced_tree.remove_node(31)
dist_gen = nx.shortest_path_length(balanced_tree)

node_distances = {}
for node, distance_dict in dist_gen:
    node_distances[node] = distance_dict
    
print(find_farthest(node_distances, marked))

31
1
32
